In [1]:
from mia_models import train_target_model, load_shadow_data, train_shadow_models
from mia_models import prepare_attack_test_data, prep_validation_data, prep_attack_train_data
from mia_models import shokri_attack, prety_print_result
import tensorflow.keras as keras
import numpy as np
from sklearn.utils import resample
import pandas as pd
import pickle
import os
import csv
import scipy.stats as stats
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn.linear_model import LogisticRegression
from statistics import mean
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import SGD
from keras.regularizers import l2
from sklearn.metrics import accuracy_score
MODEL_PATH = './model/'
DATA_PATH = './data/'

2023-06-06 12:41:46.108846: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-06-06 12:41:46.109243: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
# --------------------------------------------Original Data--------------------------------------------------------------#

In [3]:
# original_dataset=pd.read_csv('data/polish_original.csv', na_values=["?"])
# original_dataset['income_cat'] = pd.cut(original_dataset.income, bins=[0, 1000, 20000],labels=['<=1K','>1K'], include_lowest=True)
# original_dataset.drop('income', axis=1, inplace=True)
# original_dataset.drop('income_categorical', axis=1, inplace=True)
# #original_dataset.drop('smoke', axis=1, inplace=True)
# original_dataset.rename(columns = {'income_cat':'income'}, inplace = True)
# original_dataset['income'].value_counts()
# original_dataset.to_csv('data/polish_original.csv', index=False)

In [2]:
dataset_name = 'polish'
train_size = 100
attack_test_size = 50
test_start = 100
data=pd.read_csv('data/Polish/polish_original.csv', na_values=["?"], header=None).iloc[1: , :]
target_dataset = data.sample(n = 150, replace = False)
df_rest = data.loc[~data.index.isin(target_dataset.index)]
shadow_dataset = df_rest.sample(n = 2000, replace = False)
df_rest = df_rest.loc[~df_rest.index.isin(shadow_dataset.index)]
attack_test_nonmembers = df_rest.sample(n = attack_test_size, replace = False)
attack_test_members =  target_dataset.iloc[:train_size,:].sample(n = attack_test_size, replace = False)

In [3]:
# trian target model
per_class_sample=5000
channel=0   
epochs=200
n_class = 2
is_synthetic = False
VERBOSE = 0
test_ratio = 0.3

target_model, dim = train_target_model(target_dataset, dataset_name, per_class_sample, epochs, n_class, train_size, test_start, is_synthetic)

2023-06-06 12:42:13.696783: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-06-06 12:42:13.698322: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2023-06-06 12:42:13.698701: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (LAPTOP-BP5MTC3R): /proc/driver/nvidia/version does not exist
2023-06-06 12:42:13.700678: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.



 Target Train acc :  100.0 Target Test acc :  60.00000238418579


In [ ]:
#train shadow model
n_shadow_models = 20
shadow_data_size = 1200

load_shadow_data(shadow_dataset, dataset_name, n_shadow_models, shadow_data_size, test_ratio, is_synthetic)
n_shadow_train_performance, n_shadow_test_performance, n_attack_data, x_shadow_train, y_shadow_train, x_shadow_test, y_shadow_test, shadow_model_init, shadow_accuracy = train_shadow_models(dataset_name, n_shadow_models, n_class, dim, epochs, channel)

In [6]:
#train attack model
attack_test_data = prepare_attack_test_data(dataset_name, attack_test_members, attack_test_nonmembers, target_model, is_synthetic)
mem_validation, nmem_validation = prep_validation_data(attack_test_data)
attack_train_df = prep_attack_train_data(n_attack_data)
pred_membership, ori_membership, TP_idx_list, TN_idx_list, mpred, nmpred = shokri_attack(attack_train_df, mem_validation, nmem_validation, epochs)
tp, fp, fn, tn, precision, advj, acc, recall = prety_print_result (ori_membership,pred_membership)
print('Accuracy: ', acc, 'Precision: ', precision)

2/2 [==============================] - 0s 7ms/step
0
1/1 [==============================] - 0s 35ms/step
1
1/1 [==============================] - 0s 34ms/step
TP: 36     FP: 25     FN: 14     TN: 25
PPV: 0.5902
Advantage: 0.2200
Accuracy:  0.61 Precision:  0.5901639344262295


In [ ]:
# --------------------------------------------Synthetic Data------------------------------------------------------------#

In [23]:
#synthetic_dataset=pd.read_csv('data/Polish/polish_sds_cac_2500.csv', na_values=["?"])
#synthetic_dataset['income_cat'] = pd.cut(synthetic_dataset.income, bins=[0, 1000, 20000],labels=['<=1K','>1K'], include_lowest=True)
#synthetic_dataset.drop('income', axis=1, inplace=True)
#synthetic_dataset.drop('smoke', axis=1, inplace=True)
#synthetic_dataset.rename(columns = {'income_cat':'income'}, inplace = True)
#synthetic_dataset['income'].value_counts()
#synthetic_dataset.to_csv('data/Polish/polish_sds_cac_2500_mod.csv', header=False, index=False)

In [5]:
# synthetic dataset
dataset_name = 'polish'
train_size = 840
attack_test_size = 600
test_start = 840
org_dataset = pd.read_csv('data/Polish/polish_original.csv', na_values=["?"], header=None).iloc[1: , :]
target_dataset = pd.read_csv('data/Polish/polish_sds_cac_2500_mod.csv', na_values=["?"], header=None)
df = org_dataset.iloc[2500:,:]
shadow_dataset = df.sample(n = 1800, replace = False)
df_rest = df.loc[~df.index.isin(shadow_dataset.index)]
attack_test_nonmembers = df_rest.sample(n = attack_test_size, replace = False)
attack_test_members = org_dataset.iloc[:2500,:]
attack_test_members = attack_test_members.sample(n=attack_test_size, replace=False)

In [7]:
# trian target model
per_class_sample=5000
channel=0   
epochs=200
act_layer=3
n_class = 2
is_synthetic = False
VERBOSE = 0
test_ratio = 0.3

target_model, dim = train_target_model(target_dataset, dataset_name, per_class_sample, epochs, n_class, train_size, test_start, is_synthetic)


 Target Train acc :  92.02380776405334 Target Test acc :  59.638553857803345


In [27]:
#train shadow model
n_shadow_models = 20
shadow_data_size = 1200

load_shadow_data(shadow_dataset, dataset_name, n_shadow_models, shadow_data_size, test_ratio, is_synthetic)
n_shadow_train_performance, n_shadow_test_performance, n_attack_data, x_shadow_train, y_shadow_train, x_shadow_test, y_shadow_test, shadow_model_init, shadow_accuracy = train_shadow_models(dataset_name, n_shadow_models, n_class, dim, epochs, channel)

Shadow Model  0
Training shadow model 0
Shadow Train acc :  98.57142567634583 Shadow Test acc :  60.00000238418579
12/12 [==============================] - 0s 2ms/step
Shadow Model  1
Training shadow model 1
Shadow Train acc :  99.16666746139526 Shadow Test acc :  60.83333492279053
12/12 [==============================] - 0s 2ms/step
Shadow Model  2
Training shadow model 2
Shadow Train acc :  98.45238327980042 Shadow Test acc :  58.888888359069824
12/12 [==============================] - 0s 2ms/step
Shadow Model  3
Training shadow model 3
Shadow Train acc :  99.28571581840515 Shadow Test acc :  60.83333492279053
12/12 [==============================] - 0s 2ms/step
Shadow Model  4
Training shadow model 4
Shadow Train acc :  98.8095223903656 Shadow Test acc :  58.05555582046509
12/12 [==============================] - 0s 2ms/step
Shadow Model  5
Training shadow model 5
Shadow Train acc :  98.69047403335571 Shadow Test acc :  61.666667461395264
12/12 [==============================] - 0s 

In [28]:
#train attack model
attack_test_data = prepare_attack_test_data(dataset_name, attack_test_members, attack_test_nonmembers, target_model, is_synthetic)
mem_validation, nmem_validation = prep_validation_data(attack_test_data)
attack_train_df = prep_attack_train_data(n_attack_data)
pred_membership, ori_membership, TP_idx_list, TN_idx_list, mpred_syn, nmpred_syn = shokri_attack(attack_train_df, mem_validation, nmem_validation, epochs)
tp, fp, fn, tn, precision, advj, acc, recall = prety_print_result (ori_membership,pred_membership)
print('Accuracy: ', acc, 'Precision: ', precision)

19/19 [==============================] - 0s 2ms/step
0
10/10 [==============================] - 0s 1ms/step
1
10/10 [==============================] - 0s 1ms/step
TP: 337     FP: 299     FN: 263     TN: 301
PPV: 0.5299
Advantage: 0.0633
Accuracy:  0.5316666666666666 Precision:  0.529874213836478


In [ ]:
# --------------------------------------------Synthetic Data-------------------------------------------------------------#